In [1]:
#Importing the libraries
import cv2
import requests
from ultralytics import YOLO 
import matplotlib as plt
import os

In [2]:
# Load YOLO11n model
model_path = os.path.join("..", "models", "best.pt")
model = YOLO(model_path)

In [ ]:
# Telegram Bot Details
BOT_TOKEN = os.getenv("BOT_TOKEN")  # Replace with your Telegram bot token
CHAT_ID = os.getenv("CHAT_ID") # Replace with your Telegram chat ID

In [ ]:
#Telegram related functions for message delivery
def send_telegram_alert(message):
    """Send a Telegram message"""
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    response = requests.post(url, json=payload)
    print("✅ Telegram alert sent!", response.json())

In [ ]:
image_path = "img_5.jpg"  # Change this to your image
output_path = "test and output data/output images/output_tomato.jpg"
image = cv2.imread(image_path)

# Run YOLOv8 detection
results = model(image)

# Extract tomato detections (Assuming class 0 = Tomato)
tomato_count = 0
for r in results:
    for box in r.boxes:
        class_id = int(box.cls[0])  # Class index
        confidence = float(box.conf[0])  # Confidence score
        if class_id == 0 and confidence >= 0.3:  # Filter tomatoes with confidence > 30%
            tomato_count += 1
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw box
            cv2.putText(image, f"Tomato", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display count on image
cv2.putText(image, f"Tomatoes: {tomato_count}", (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Save output image
cv2.imwrite(output_path, image)
print(f"✅ Output saved as {output_path}")

# Send Telegram alert if tomatoes are less than 5
if tomato_count < 5:
    alert_message = f"⚠️ Low Tomato Alert! Only {tomato_count} detected."
    send_telegram_alert(alert_message)


In [ ]:
cv2.imshow("Tomato Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()